In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
spark

In [4]:
import pyspark.sql.types as t
import pyspark.sql.functions as f
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, FloatType, LongType, ArrayType
from pyspark.ml.feature import Tokenizer, HashingTF, StringIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier

In [5]:
! hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2022-01-06 18:46 /labs/slaba04/gender_age_dataset.txt


In [6]:
! hdfs dfs -cat /labs/slaba04/gender_age_dataset.txt | head -n2

gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
cat: Unable to write to output stream.


In [7]:
path = '/labs/slaba04/gender_age_dataset.txt'

schema = StructType([
    StructField("gender", StringType()),
    StructField("age", StringType()),
    StructField("uid", StringType()),
    StructField("user_json", StringType()),
])

train_data = spark.read.csv(path, header=True, schema=schema, sep='\t')

In [8]:
train_data.show(2)

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|           user_json|
+------+-----+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|
+------+-----+--------------------+--------------------+
only showing top 2 rows



In [9]:
train_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)



In [10]:
visits_schema = StructType([
    StructField("visits", ArrayType(
        StructType([
            StructField("url", StringType(), True),
            StructField("timestamp", LongType(), True)
        ])
    ))    
])

In [11]:
# Фичи
df = train_data.withColumn("visits",f.from_json(train_data.user_json, schema=visits_schema))        

In [12]:
df = df.withColumn('urls' , df['visits']['visits']['url'] )


In [13]:
df = df.withColumn("host", f.expr('transform(urls, x -> parse_url(x, "HOST" ))'))

In [14]:
df.show(5)

+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|gender|  age|                 uid|           user_json|              visits|                urls|                host|
+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|[[[http://zebra-z...|[http://zebra-zoy...|[zebra-zoya.ru, n...|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|[[[http://sweetra...|[http://sweetradi...|[sweetrading.ru, ...|
|     F|25-34|d50237ea-747e-48a...|{"visits": [{"url...|[[[http://ru.orif...|[http://ru.orifla...|[ru.oriflame.com,...|
|     F|25-34|d502f29f-d57a-46b...|{"visits": [{"url...|[[[http://transla...|[http://translate...|[translate-tattoo...|
|     M| >=55|d503c3b2-a0c2-4f4...|{"visits": [{"url...|[[[https://mail.r...|[https://mail.ram...|[mail.rambler.ru,...|
+------+-----+--------------------+-----

In [15]:
df.groupby('age').count().show()

+-----+-----+
|  age|count|
+-----+-----+
| >=55| 1679|
|45-54| 4744|
|    -| 5000|
|35-44| 9360|
|25-34|15457|
|18-24| 4898|
+-----+-----+



In [16]:
train = df.filter("age != '-'").filter("gender != '-'").drop('visits', 'urls', 'user_json')

In [17]:
train.show()

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|                host|
+------+-----+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|[zebra-zoya.ru, n...|
|     M|25-34|d502331d-621e-472...|[sweetrading.ru, ...|
|     F|25-34|d50237ea-747e-48a...|[ru.oriflame.com,...|
|     F|25-34|d502f29f-d57a-46b...|[translate-tattoo...|
|     M| >=55|d503c3b2-a0c2-4f4...|[mail.rambler.ru,...|
|     F|25-34|d5090ddf-5648-487...|[cfire.mail.ru, p...|
|     F|25-34|d50bcef8-16ff-4e8...|[www.msn.com, www...|
|     F|18-24|d50e23dc-0cbd-488...|[www.gazprom.ru, ...|
|     F|45-54|d50fdabb-4208-441...|[lifenews.ru, lif...|
|     F|18-24|d511b480-23a6-482...|[www.google.ru, f...|
|     F|25-34|d51294ed-1b95-4e4...|[muz4in.net, www....|
|     F|25-34|d512e295-6a85-491...|[kosmetista.ru, k...|
|     M|25-34|d51441ea-9dda-454...|[android.mobile-r...|
|     F|25-34|d51822d4-105b-457...|[tsn.ua, cfts.org...|
|     F|35-44|d5183db2-c8e5-413

In [18]:
#pipeline
hashing_TF = HashingTF(inputCol="host", outputCol="features", numFeatures=10000)

indexer_age = StringIndexer(inputCol="age", outputCol="age_label").fit(train)
indexer_gender = StringIndexer(inputCol="gender", outputCol="gender_label").fit(train)

rfc_age = RandomForestClassifier(featuresCol="features", labelCol="age_label", predictionCol="age_label_pred", 
        rawPredictionCol="age_label_pred_raw", probabilityCol="age_label_pred_prob")

rfc_gender = RandomForestClassifier(featuresCol="features", labelCol="gender_label", predictionCol="gender_label_pred",
                rawPredictionCol="gender_label_raw_pred", probabilityCol="gender_label_prob")

In [19]:
pipeline = Pipeline(stages=[hashing_TF, 
                            indexer_age, 
                            indexer_gender, 
                            rfc_age, 
                            rfc_gender
                           ])

In [20]:
model = pipeline.fit(train)

In [21]:
sample1 = df.sample(True, 0.1, seed=0)

In [22]:
pred = model.transform(sample1)

In [24]:
pred.show(1)

+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------------+--------------------+--------------------+--------------+---------------------+--------------------+-----------------+
|gender|  age|                 uid|           user_json|              visits|                urls|                host|            features|age_label|gender_label|  age_label_pred_raw| age_label_pred_prob|age_label_pred|gender_label_raw_pred|   gender_label_prob|gender_label_pred|
+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------------+--------------------+--------------------+--------------+---------------------+--------------------+-----------------+
|     F|25-34|d512e295-6a85-491...|{"visits": [{"url...|[[[http://kosmeti...|[http://kosmetist...|[kosmetista.ru, k...|(10000,[8726,9157...|      0.0|    

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [28]:
evaluator = MulticlassClassificationEvaluator(labelCol="age_label",
                                                  predictionCol="gender_label_pred", metricName="f1")

In [30]:
model.write().overwrite().save("/user/inessa.zabelina/labs/lab4_model")

In [ ]:
# df = df.withColumn("event_properties", f.explode(f.array("visits.*")))
# df_exp = df.withColumn("visit", f.explode("event_properties"))
# df_exp.printSchema()
# df_exp = df_exp.withColumn('url', f.expr('parse_url(visit["url"], "HOST")'))
# df_exp = df_exp.withColumn('timestamp', df_exp.visit["timestamp"])
# df_new = df_exp.withColumn("new_json", 
#     f.struct(*[f.col('url').alias('url'), f.col('timestamp').alias('timestamp')])
# )
# df_new.printSchema()
# df = df_new.select('gender', 'age', 'uid', 'new_json')

# df_train = df.groupby('gender', 'age', 'uid').agg(f.collect_list("new_json").alias('url_timestamp'))

In [31]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-1.newprolab.com:6667'
KAFKA_INPUT_TOPIC = 'input_inessa.zabelina'
KAFKA_OUTPUT_TOPIC = 'inessa.zabelina'

In [32]:
input_kafka_params = {
    "kafka.bootstrap.servers": "spark-master-1.newprolab.com:6667",
    "subscribe": "input_inessa.zabelina",
    "startingOffsets": "earliest",
    "maxOffsetsPerTrigger": "5"
}

In [33]:
write_kafka_params = {
    "kafka.bootstrap.servers": "spark-master-1.newprolab.com:6667",
    "topic": "inessa.zabelina"
}

In [34]:
sdf = spark.readStream.format("kafka").options(**input_kafka_params).load()

In [35]:
event_type = StructType([
    StructField("uid", StringType(), True),
    StructField("visits", StringType(), True)
])

In [36]:
visit_type = ArrayType(
    StructType([
        StructField("url", StringType(), True),
        StructField("timestamp", LongType(), True)
    ])
)

In [37]:
def foreachBatchFunction(batch_df, batch_id) :
    
    parced_data = batch_df\
        .select(col("value").cast("string").alias("value"))\
        .select(from_json(col("value"), event_type).alias("data"))\
        .select("data.*")\
        .select("uid", from_json(col("visits"), visit_type).alias("visits"))
    parced_data.show(1)
    
    proc_df = parced_data\
        .withColumn("visit", explode("visits").alias("visits"))\
        .withColumn("host", regexp_replace(expr("parse_url(visit.url), 'HOST'").alias("host")), "^www.", "")\
        .filter("age != '-'").filter("gender != '-'").drop("visits", "visit")\
        .groupBy("uid")\
        .agg(collect_list("host").alias("hosts"))
    
    proc_df.show(1)
    
    predict_df = lab_model\
        .transform(proc_df)\
        .select("uid", "PredictedGender", "PredictedAge")\
        .withColumnRenamed("PredictedAge", "age")\
        .withColumnRenamed("PredictedGender", "gender")
    
    predict_df.show(1)
    
    predict_df = predict_df.select(to_json(struct(*predict_df.columns)).alias("value"))

    return predict_df


In [38]:
lab_model = PipelineModel.load("/user/inessa.zabelina/labs/lab4_model")

In [39]:
sink = sdf.writeStream\
    .foreachBatch(foreachBatchFunction)\
    .format('kafka')\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)\
    .option("topic", KAFKA_OUTPUT_TOPIC)\
    .option("checkpointLocation", 'streaming/chk/chk_kafka')\
    .outputMode("append")

In [40]:
sq = sink.start()

In [41]:
sq.isActive

True

In [42]:
sq.status

{'message': 'Writing offsets to log',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [43]:
sq.recentProgress

[{'id': 'ba1dd17e-31f7-4106-b520-f301924999af',
  'runId': 'cfebd165-bbbf-4244-8239-632332653baa',
  'name': None,
  'timestamp': '2023-02-27T19:34:56.382Z',
  'batchId': 4034,
  'numInputRows': 5,
  'inputRowsPerSecond': 48.07692307692308,
  'processedRowsPerSecond': 50.505050505050505,
  'durationMs': {'addBatch': 31,
   'getBatch': 0,
   'getEndOffset': 1,
   'queryPlanning': 7,
   'setOffsetRange': 1,
   'triggerExecution': 99,
   'walCommit': 31},
  'stateOperators': [],
  'sources': [{'description': 'KafkaV2[Subscribe[input_inessa.zabelina]]',
    'startOffset': {'input_inessa.zabelina': {'0': 25165}},
    'endOffset': {'input_inessa.zabelina': {'0': 25170}},
    'numInputRows': 5,
    'inputRowsPerSecond': 48.07692307692308,
    'processedRowsPerSecond': 50.505050505050505}],
  'sink': {'description': 'org.apache.spark.sql.kafka010.KafkaSourceProvider@62f8529a'}},
 {'id': 'ba1dd17e-31f7-4106-b520-f301924999af',
  'runId': 'cfebd165-bbbf-4244-8239-632332653baa',
  'name': None,
 

In [ ]:
# kafka_read_df = (spark.readStream
#                  .format("kafka")
#                  .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
#                  .option("subscribe", KAFKA_INPUT_TOPIC)
#                  .option("startingOffsets", "earliest")
#                  .option("failOnDataLoss", "False")
#                  .load()
# )

In [ ]:
# event_schema = StructType([
#     StructField("uid", StringType(), True),
#     StructField("visits", StringType(), True),
# ])

# visits_schema = ArrayType(
#         StructType([
#             StructField("url", StringType(), True),
#             StructField("timestamp", LongType(), True)
#         ])
# )

In [ ]:
# clean_df = (
#     kafka_read_df
#     .select(f.col('value').cast('string').alias('value'))
#     .select(f.from_json(f.col('value'), event_schema).alias('event'))
#     .select('event.uid',f.from_json(f.col('event.visits'), visits_schema).alias('visits'))
# )


In [ ]:
# lab_model = PipelineModel.load("/user/inessa.zabelina/labs/lab4_model")

In [ ]:
#извлечение url
#применение модели, сохранение предсказаний в predictions_df

# proc_df = clean_df\
#     .withColumn("visit", f.explode("visits").alias("visits"))\
#     .withColumn("host", 
#                 regexp_replace(expr("parse_url(visit.url), 'HOST'").alias("host")), "^www.", "")\
#     .filter("age != '-'")\
#     .filter("gender != '-'")\    
#     .drop("visits", "visit")\
#     .groupBy("uid")\
#     .agg(collect_list("host").alias("hosts"))
    
# predictions_df = lab_model\
#     .transform(proc_df)\
#     .select("uid", "PredictedGender", "PredictedAge")\
#     .withColumnRenamed("PredictedAge", "age")\
#     .withColumnRenamed("PredictedGender", "gender")


In [ ]:
# #Оборачивание предсказания обратно в json
# kafka_out_df = (

#     predictions_df
#     .select(f.to_json(f.struct(*predictions_df.columns)).alias('value'))
# )


# #запись в выходной топик

# sink = (
#     kafka_out_df
#     .writeStream
#     .format('kafka')
#     .options("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
#     .options("topic", KAFKA_OUTPUT_TOPIC)
#     .option("checkpointLocation", 'streaming/chk/chk_kafka')
#     .outputMode("append")
# )



In [ ]:
# sq = sink.start()

In [45]:
def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    for s in streams:
        desc = s.lastProgress["sources"][0]["description"]
        s.stop()
        print("Stopped {s}".format(s=desc))

In [46]:
kill_all()

Stopped KafkaV2[Subscribe[input_inessa.zabelina]]


In [47]:
sc.stop()